In [4]:
from fontTools.ttLib.woff2 import bboxFormat
!python --version

Python 3.11.11


In [6]:
import torch

torch.__version__

'2.6.0'

In [7]:
import torch

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print("MPS device not found.")

tensor([1.], device='mps:0')


In [8]:
import ultralytics

ultralytics.__version__

'8.3.78'

# Auto-annotate Images

In [10]:
from ultralytics.data.annotator import auto_annotate
auto_annotate(data="/Users/chen/Downloads/", det_model="yolo11x.pt", sam_model="sam2_b.pt", classes=[41])


image 1/273 /Users/chen/Downloads/project_image_baidu_1.jpg: 640x480 1 cup, 306.0ms
image 2/273 /Users/chen/Downloads/project_image_baidu_10.jpg: 640x480 1 cup, 457.3ms
image 3/273 /Users/chen/Downloads/project_image_baidu_100.jpg: 448x640 (no detections), 252.2ms
image 4/273 /Users/chen/Downloads/project_image_baidu_101.jpg: 640x480 2 cups, 270.6ms
image 5/273 /Users/chen/Downloads/project_image_baidu_102.jpg: 416x640 2 cups, 243.6ms
image 6/273 /Users/chen/Downloads/project_image_baidu_103.jpg: 640x480 1 cup, 270.6ms
image 7/273 /Users/chen/Downloads/project_image_baidu_104.jpg: 384x640 1 cup, 219.5ms
image 8/273 /Users/chen/Downloads/project_image_baidu_105.jpg: 640x480 1 cup, 270.2ms
image 9/273 /Users/chen/Downloads/project_image_baidu_106.jpg: 448x640 1 cup, 310.8ms
image 10/273 /Users/chen/Downloads/project_image_baidu_107.jpg: 640x640 1 cup, 344.4ms
image 11/273 /Users/chen/Downloads/project_image_baidu_108.jpg: 640x640 1 cup, 327.8ms
image 12/273 /Users/chen/Downloads/project

# Segment to Bounding Box Conversion

In [11]:
import os
import numpy as np
from ultralytics.utils.ops import segments2boxes


def process_files_in_folder(input_folder, output_folder):
    # Ensure the output folder exists or create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all the files in the folder
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        print(file_path)
        # Ensure we only process files (and not directories)
        if os.path.isfile(file_path) and file_path.endswith('.txt'):
            with open(file_path, 'r') as f:
                # Generate an output file name and write the processed output
                output_file_name = file_name
                output_file_path = os.path.join(output_folder, output_file_name)
                # Read the content of the file and process it
                content = f.read()
                curr = []
                with open(output_file_path, 'w') as out_file:
                    for i in content.split():
                        if i == '41':
                            if curr != []:
                                # print(curr)
                                segments = np.array([[curr]]).astype(float)
                                bbox = segments2boxes([s.reshape(-1, 2) for s in segments])
                                out_file.write("41 ")
                                # print(list(bbox[0]))
                                for box in list(bbox[0]):
                                    out_file.write(str(box))
                                    out_file.write(' ')
                                out_file.write('\n')
                                print(list(bbox[0]))
                            curr = []
                            continue
                        curr.append(i)
                    segments = np.array([[curr]]).astype(float)
                    bbox = segments2boxes([s.reshape(-1, 2) for s in segments])
                    out_file.write("41 ")
                    # print(list(bbox[0]))
                    for box in list(bbox[0]):
                        out_file.write(str(box))
                        out_file.write(' ')
                    out_file.write('\n')
                    print(list(bbox[0]))


input_folder = '/Users/chen/Downloads_auto_annotate_labels'
output_folder = '/Users/chen/Downloads_auto_annotate_labels/bbox'

process_files_in_folder(input_folder, output_folder)

/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_187.txt
[0.443, 0.4481982, 0.598, 0.78228223]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_193.txt
[0.525, 0.56071967, 0.666, 0.8125937]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_178.txt
[0.538, 0.60194904, 0.42799997, 0.3763118]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_150.txt
[0.5, 0.515, 0.28266668, 0.706]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_7.txt
[0.49199998, 0.47305387, 0.648, 0.7724551]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_232.txt
[0.49250937, 0.51, 0.22097376, 0.63600004]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_36.txt
[0.44736844, 0.5578947, 0.22105265, 0.47017545]
[0.31710526, 0.4350877, 0.21315789, 0.44210523]
[0.8013158, 0.5350877, 0.23421049, 0.44561404]
[0.6421052, 0.41228068, 0.19999999, 0.41754386]
/Users/chen/Downloads_auto_annotate_labels/project_image_baidu_22.txt
[0.52773

# Visualize Bounding Boxes

In [12]:
# coding:utf-8
import cv2
import os
import random

In [27]:
def plot_one_box(x, image, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(
        0.002 * (image.shape[0] + image.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    # print(x)
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(image, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(image, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(image, label, (c1[0], c1[1] - 2), 0, tl / 3,
                    [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

In [28]:
input_folder = 'data/man/labels'
for file_name in os.listdir(input_folder):
    if file_name.endswith('.txt'):
        image_name = file_name.split('.')[0]
        try:
            img = cv2.imread('data/man/images/' + image_name + '.jpg')
            height, width = img.shape[:2]
        except:
            print(image_name)
            continue

        with open(os.path.join(input_folder, file_name), 'r') as file:
            # Iterate through each line in the file
            for line in file:
                # Strip any leading/trailing whitespace and then split the line by spaces
                staff = line.strip().split()
                # Print the result of the split operation
                x_center, y_center, w, h = float(staff[1]) * width, float(staff[2]) * height, float(
                    staff[3]) * width, float(staff[4]) * height
                x1 = round(x_center - w / 2)
                y1 = round(y_center - h / 2)
                x2 = round(x_center + w / 2)
                y2 = round(y_center + h / 2)
                if staff[0] == '0':
                    plot_one_box([x1, y1, x2, y2], img, color=[0, 255, 0], label='Cup')
                else:
                    plot_one_box([x1, y1, x2, y2], img, color=[0, 0, 255], label='Timmies')
                # plot_one_box([x1, y1, x2, y2], img)
                cv2.imwrite('data/man/bbox/' + image_name + '.jpg', img)
